In [ ]:
# Import all libraries
import itertools
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy
import keras
import tensorflow
import os
from keras.models import Sequential
from keras.layers import *
from keras.layers.advanced_activations import PReLU
from keras.layers.advanced_activations import ThresholdedReLU
from keras.optimizers import Adam
from matplotlib import pyplot as pyplot
import tensorflow as tf
from sklearn.model_selection import train_test_split
import sklearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE



In [ ]:
train_data = pd.read_csv('train.csv')
print(train_data)
test_data = pd.read_csv('test.csv')
print(test_data)

In [ ]:
# dividing data into features and labels for training data
label=train_data['target']
train_name = [col for col in train_data if (col.startswith('var_'))]
train_features=pd.DataFrame(train_data, columns = train_name)

# dividing data into features and labels for testing data
test_name = [col for col in train_data if (col.startswith('var_'))]
test_features=pd.DataFrame(train_data, columns = test_name)



In [ ]:
#Oversampling the unbalanced dataset
X_train, X_test, y_train, y_test = train_test_split(train_features, label, test_size=0.3, random_state=0)

print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())


print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))


In [ ]:
#Fitting a logistic regression to the model
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train_res, y_train_res)

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
#Plotting the ROC curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()


In [ ]:
# Running a FC NN on the dataset
    model = Sequential()   
    model.add(Dense(200, bias_initializer='zeros', activation='relu',input_dim=200))
    model.add(Dense(150, bias_initializer='zeros', activation='relu'))
    model.add(Dense(100, bias_initializer='zeros', activation='relu'))
    model.add(Dense(50, bias_initializer='zeros', activation='relu'))
    model.add(Dense(15, bias_initializer='zeros', activation='relu'))
    model.add(Dense(1, bias_initializer='zeros', activation='sigmoid'))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    
    # fit
history=model.fit(X_train,
        y_train, epochs=20, batch_size=500)
        
    # save weights
    model.save('model.dat')

In [ ]:
  #score = model.evaluate(X_test, y_test, batch_size=10)
    predictions = model.predict(test_features, batch_size=10)
   # print("evaluation score:", score)

In [ ]:
columns = ['ID_code','target']
output = pd.DataFrame(columns=columns)
output['ID_code']=test_data['ID_code']
output['target']=predictions
print(output)
output.to_csv('santander', index=False)

In [ ]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc
y_pred = model.predict_proba(X_test)
roc_auc_score(y_test, y_pred)

In [ ]:
# Testing a random forest classifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200, max_depth=2,random_state=0)
clf.fit(X_train,y_train)


In [ ]:
pred=clf.predict(X_test)
roc_auc_score(y_test, pred)